In [4]:
import os

def find_txt_with_most_words(folder_path):
    most_words_count = 0
    file_with_most_words = ""

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                word_count = len(content.split())
                if word_count > most_words_count:
                    most_words_count = word_count
                    file_with_most_words = file_name

    if file_with_most_words:
        print(f"The file '{file_with_most_words}' has the most words: {most_words_count} words.")
    else:
        print("No .txt files found in the folder.")

folder_path = "."  # Refers to the current folder, you can change it as needed.
find_txt_with_most_words(folder_path)


The file '1645041228778.2522.txt' has the most words: 360 words.


In [7]:
import os
import re

def find_longest_word_ignore_tags(folder_path):
    longest_word = ""
    longest_word_length = 0
    longest_word_file = ""

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                # Remove tags
                content = re.sub(r'<start_".*?">', '', content)
                content = re.sub(r'<end_".*?">', '', content)
            
                words = content.split()
                for word in words:
                    if len(word) > longest_word_length:
                        longest_word_length = len(word)
                        longest_word = word
                        longest_word_file = file_name

    if longest_word:
        print(f"The longest word ignoring tags in the files is '{longest_word}' in the file '{longest_word_file}', with a length of {longest_word_length}.")
    else:
        print("No .txt files found in the folder.")

folder_path = "."  # Refers to the current folder, you can change it as needed.
find_longest_word_ignore_tags(folder_path)


The longest word ignoring tags in the files is '###############***************************' in the file '1701069672189.122.txt', with a length of 42.


In [13]:
import os
import re
from collections import defaultdict

def find_longest_words_ignore_tags(folder_path, top_n=10):
    longest_words = []
    longest_word_length = 0
    longest_word_file = ""
    total_txt_files = 0
    word_length_distribution = defaultdict(int)

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            total_txt_files += 1
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                # Remove tags
                content = re.sub(r'<start_".*?">', '', content)
                content = re.sub(r'<end_".*?">', '', content)
            
                words = content.split()
                for word in words:
                    word_length = len(word)
                    word_length_distribution[word_length] += 1
                    if word_length >= longest_word_length:
                        longest_word_length = word_length
                        longest_word = word
                        longest_word_file = file_name
                        longest_words.append((longest_word, longest_word_length, longest_word_file))
                        longest_words.sort(key=lambda x: x[1], reverse=True)
                        if len(longest_words) > top_n:
                            longest_words.pop()

    if longest_words:
        print(f"The top {top_n} longest words ignoring tags in the files are:")
        for i, (word, length, file) in enumerate(longest_words):
            print(f"{i+1}. '{word}' in the file '{file}', with a length of {length}.")

    print(f"Total number of .txt files read: {total_txt_files}")

    print("Word Length Distribution:")
    for length, count in sorted(word_length_distribution.items()):
        print(f"Length {length}: {count} words")

folder_path = "."  # Refers to the current folder, you can change it as needed.
find_longest_words_ignore_tags(folder_path)


The top 10 longest words ignoring tags in the files are:
1. '###############***************************' in the file '1701069672189.122.txt', with a length of 42.
2. 'https://sorgula.turkcellesirket.com/earsiv' in the file '1701069827728.169.txt', with a length of 42.
3. 'ETTN:910C1723-F829-48EC-8E14-DD9E4C9B12EA' in the file '1701069543694.7068.txt', with a length of 41.
4. '****************************************' in the file '1637587138269.084.txt', with a length of 40.
5. ':AB049F5E-E350-440B-BD88-CDC2DA428951' in the file '112.txt', with a length of 37.
6. ':AB049F5E-E350-440B-BD88-CDC2DA428951' in the file '133.txt', with a length of 37.
7. '72740011109111780024' in the file '110.txt', with a length of 20.
8. 'www.halktanpazar.com' in the file '112.txt', with a length of 20.
9. 'HO:000000001486268' in the file '0.txt', with a length of 18.
10. 'NO:000000000382710' in the file '1.txt', with a length of 18.
Total number of .txt files read: 1094
Word Length Distribution:
Length 1: 

In [ ]:
max_len=20

In [2]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QFileDialog, QComboBox
import easyocr
import cv2
import pytesseract
from google.cloud import vision
from google.oauth2 import service_account
import io

credentials = service_account.Credentials.from_service_account_file('key.json')

client = vision.ImageAnnotatorClient(credentials=credentials)


class ImageReader(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Image Text Reader")
        self.setGeometry(100, 100, 400, 250)

        self.layout = QVBoxLayout()

        self.image_label = QLabel("Görsel burada görünecek")
        self.layout.addWidget(self.image_label)

        self.load_button = QPushButton("Görsel Yükle")
        self.load_button.clicked.connect(self.load_image)
        self.layout.addWidget(self.load_button)

        self.ocr_selector = QComboBox()
        self.ocr_selector.addItems(["EasyOCR", "Tesseract OCR", "Google Vision API"])
        self.layout.addWidget(self.ocr_selector)

        self.text_label = QLabel("Metin burada görünecek")
        self.layout.addWidget(self.text_label)

        self.setLayout(self.layout)

    def load_image(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(self, "Görsel Yükle", "", "Image files (*.jpg *.png *.jpeg *.bmp *.gif)", options=options)
        if file_path:
            self.process_image(file_path)

    def process_image(self, file_path):
        ocr_method = self.ocr_selector.currentText()

        if ocr_method == "EasyOCR":
            img_cv = cv2.imread(file_path)
            if img_cv is not None:
                img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
                reader = easyocr.Reader(['tr'])  # Türkçe dil modellerini kullan
                results = reader.readtext(img_rgb)
                text = "\n".join([detection[1] for detection in results])
                print(text)
                self.text_label.setText(text)
            else:
                print("Görüntü yüklenemedi.")
        elif ocr_method == "Tesseract OCR":  # Tesseract OCR
            text = pytesseract.image_to_string(file_path, lang="tur")
        else:  # Google Cloud Vision
            with io.open(file_path, 'rb') as image_file:
                content = image_file.read()

            image = vision.Image(content=content)

            response = client.text_detection(image=image)
            texts = response.text_annotations

            if texts:
                text = texts[0].description
                print(text)
                self.text_label.setText(text)
            else:
                print("Metin tespit edilemedi.")

            print(text)
            self.text_label.setText(text)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ImageReader()
    window.show()
    sys.exit(app.exec_())


: 